# Classical Machine Learning

In this first Notebook, we will discuss classical but powerful machine learning model that are widely used for text classification.

The first question is, how to featurize the text. We can use words, subwords characters or even bytes to represent the our dataset (IMDB review). For English text classification, we usually use words or subwords as features. These features capture the linguistic features more than bytes or characters. In this first three Notebook, we will use words as features, and will discuss subwords in later Notebook. Note that this is not the only ways we featurize text. For other problem like language detection, we can use character n-grams as features (think of it!). In this Notebook, we will first discuss and experiment several tokenization techniques.

The second question is how to vectorize i.e. how to represent features into vector space. The simplest way is one-hot encoding, which is a vector of size equal to number of words words in consideration where all element is zero but the element corresponding to the word being represented. In section 2 of this Notebook, we will discuss and experiment several vectorization techniques.

The last question is which model. In this experiment, we will use [logistic regression](https://web.stanford.edu/~jurafsky/slp3/5.pdf) and [Naive Bayes]( https://web.stanford.edu/~jurafsky/slp3/4.pdf), which are commonly used for text classification. 



In [ ]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny
from lib.classical_ml import run_pipeline


In [2]:
dataset  = download_tfds_imdb_as_text()
tiny_dataset = download_tfds_imdb_as_text_tiny()

# Experiment 1 Tokenizer


I will use SpaCy tokenizer throughout the experiment, see [documentation](https://spacy.io/usage/linguistic-features#tokenization) to understand its algorithm.

We will try two different things besidesSpaCy tokenizer 

- Remove stopwords: In our problem, we can think that words like `is`, `of` do not carry any linguistic features that are useful for text classification. We will remove them from the text so that it will not introduce noise to the model. Moreover, these words are likely to appear in almost every document so they provide very less information for the model to classify the document. However, removing stop is not necessary useful. We can also say that a good model should be able to learn this by itself. We will see if this is useful or not.

- lower and lemma
The intuition of lowercase and lematization is that it can group words with similar meaning but in different form together. For example, `It is a good movie.` and `It is the best movie.` If we tokenize and lemmatize these two sentneces, the results will share the token `good`. If we tokenize but not lemmatize, `good` and `best` will be different tokens. Lemmatization may or may not improve the accuracy of models. It depends on what kind of NLP tasks we are wokring on. Let's do the experiment and see how lemmatization effects the accuracy of our models and discuss why.



In [3]:
# approximate running time: 42 mins
    
print("Simple SpaCy tokenizer")
_, _ = run_pipeline(dataset)

print("Simple SpaCy tokenizer and ignore stop")
_, _ = run_pipeline(dataset, lower=True, ignore=["like_num", "is_stop"])

print("Simple SpaCy tokenizer, lowercase, lemma")
_, _ = run_pipeline(dataset, lower=True, lemma=True)
    


Simple SpaCy tokenizer
Best parameters set found on development set:  {'model__C': 0.1}
Best F1 on development set: 0.89 2
F1 on test set: 0.89
Simple SpaCy tokenizer and ignore stop
Best parameters set found on development set:  {'model__C': 0.1}
Best F1 on development set: 0.88 2
F1 on test set: 0.87
Simple SpaCy tokenizer, lowercase, lemma
Best parameters set found on development set:  {'model__C': 0.1}
Best F1 on development set: 0.89 2
F1 on test set: 0.88


TODO

# Experiment 2 Vectorizer

Prerequisite: If you are not familiar with TFIDF, read see [this](https://nlp.stanford.edu/IR-book/pdf/06vect.pdf).

In this experiement, we will try different vectorization techniques; Bigrams, TFIDF and Binary. Although all of them are based one hot encoding, they capture slightly different information from text. 

- Bigrams: TODO `This movie is not good. It is boring` vs `This movie is not boring. It is good`

- TFIDF: TFIDF is very common technique for Information Retrieval (IR) and has been long proved that it improves the IR performance. However, text classification and IR are two different problem sets, so this is not neccessary the case. For text classification, we can expect that whatever classification models we use can capture the same things as TFIDF, which is how importance words are, or how much information words contain.

- Binary: TODO (treat `good good good` and `good` similarly)


In [4]:
# approximate running time: 82 mins
    
print("Simple SpaCy tokenizer")
_, _ = run_pipeline(dataset, lower=True, lemma=True, bigram=True)

print("Simple SpaCy tokenizer and ignore stop")
_, _ = run_pipeline(dataset, lower=True, lemma=True, tfidf=True)

print("Simple SpaCy tokenizer, lowercase, lemma")
_, _ = run_pipeline(dataset, lower=True, lemma=True, binary=True)
    


Simple SpaCy tokenizer
Best parameters set found on development set:  {'model__C': 0.1}
Best F1 on development set: 0.90 2
F1 on test set: 0.90
Simple SpaCy tokenizer and ignore stop
Best parameters set found on development set:  {'model__C': 10}
Best F1 on development set: 0.89 2
F1 on test set: 0.88
Simple SpaCy tokenizer, lowercase, lemma
Best parameters set found on development set:  {'model__C': 0.1}
Best F1 on development set: 0.89 2
F1 on test set: 0.88


TODO TODO TODO

# Experiment 3 Model

Both Naive Bayes and Logistic Regression are classification model. However, they are based on very different mathematical notion. Naive Bayes is generative model which you predict targets `y` by joint probability `P(XY)`. We estimate the parameters of model by maximum likelihood technique, which basically counting the number of occurrence of samples of each cases. On the other hand, logistic regression is discriminative model which predict targets directly `P(Y|X)`. The parameters of the model are estimated with numerical methods i.e. gradient descent.

Why this is important? You would rather understand the mathematical notion of the model rather than think of them as  black boxes. These two models can introduce different problems that can be solved differently. For example
- Logistic Regression can have convergence problem, Naive Bayes don't
- Naive Bayes can give 0 probability if one of the features has not been seen in a class
- Logistic Regression result is indeterministic. It can have different result in different training
- Naive Bayes can be biased if number of samples in class A is larger than class B
- etc

Fortunately, these possible problems do not present in experiment shown below. But you have to keep in mind that this can happen. 


In [5]:
# approximate running time: 3 mins

import time
now = time.time()
print("Simple NB")
_, _ = run_pipeline(dataset, use_nb=True)
    
    
print(time.time()-now)

Simple NB
Best parameters set found on development set:  {}
Best F1 on development set: 0.84 2
F1 on test set: 0.80
167.90781021118164


We observe much lower performance from Naive Bayes. In my experience working on text classification, I rarely see the case when Naive Bayes is better than logistic regression.